In [6]:
import numpy as numpy
import json
import os
import pandas as pd
import urllib.request

In [ ]:
import requests
import pandas as pd
import time
import os

def get_binance_ohlc(symbol, interval, start_ts, end_ts):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_ts,
        "endTime": end_ts,
        "limit": 1000
    }
    response = requests.get(url, params=params)
    data = response.json()
    return pd.DataFrame(data)

# ✅ Create output directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# ✅ Settings
coins = ["BTCUSDT", "ETHUSDT", "LTCUSDT", "XMRUSDT"]
interval = "5m"
start_times = {
    "BTCUSDT": "2018-01-01",
    "ETHUSDT": "2018-01-01",
    "LTCUSDT": "2018-01-01",
    "XMRUSDT": "2020-11-19"  # ✅ XMR listed late
}
end_time = pd.Timestamp("2021-01-02")
end_ms = int(end_time.timestamp() * 1000)
step = 1000 * 5 * 60 * 1000  # 1000 candles * 5 min * 60 sec * 1000 ms

# ✅ Loop through coins
for symbol in coins:
    print(f"\nFetching data for: {symbol}")
    start_str = start_times[symbol]
    start = int(pd.Timestamp(start_str).timestamp() * 1000)
    all_data = []

    while start < end_ms:
        end = min(start + step, end_ms)
        df = get_binance_ohlc(symbol, interval, start, end)

        if df.empty:
            print("No more data.")
            break

        df[0] = pd.to_datetime(df[0], unit='ms')
        all_data.append(df)

        start = int(df[0].iloc[-1].timestamp() * 1000) + 1
        time.sleep(0.1)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.columns = [
            "Open Time", "Open", "High", "Low", "Close", "Volume",
            "Close Time", "Quote Volume", "Number of Trades",
            "Taker Buy Base", "Taker Buy Quote", "Ignore"
        ]
        final_df.to_csv(f"data/{symbol}_2018.csv", index=False)
        print(f"Saved: data/{symbol}_2018.csv | Rows: {final_df.shape[0]}")
    else:
        print(f"No data collected for {symbol}.")


AttributeError: 'dict' object has no attribute 'timestamp'